In [76]:
import sys
import json
import time
import logging
import twitter
import urllib.parse

from os import environ as e

t = twitter.Api(
    consumer_key="Z1NihAjRxYkMOvBqBduU7DVzk",
    consumer_secret="sMI1MTaigTXGxPsy8UYxmvls09xBVt8Ee4djJbiZB7jH2Gtrew",
    access_token_key="994429576557326336-dpBzm8r9gLQRymZJUYnoXsDlmz9m6IR",
    access_token_secret="eqLHK9F9GfZ02hvB3R0fJoOF8CanR6PiIzWouXCQz1Xlf",
    sleep_on_rate_limit=True
)

def tweet_url(t):
    return "https://twitter.com/%s/status/%s" % (t.user.screen_name, t.id)

def get_tweets(filename):
    for line in open(filename):
        yield twitter.Status.NewFromJsonDict(json.loads(line))

def get_replies(tweet):
    user = tweet.user.screen_name
    tweet_id = tweet.id
    max_id = None
    logging.info("looking for replies to: %s" % tweet_url(tweet))
    while True:
        q = urllib.parse.urlencode({"q": "to:%s" % user})
        try:
            replies = t.GetSearch(raw_query=q, since_id=tweet_id, max_id=max_id, count=100)
        except twitter.error.TwitterError as e:
            logging.error("caught twitter api error: %s", e)
            time.sleep(60)
            continue
        for reply in replies:
            logging.info("examining: %s" % tweet_url(reply))
            if reply.in_reply_to_status_id == tweet_id:
                logging.info("found reply: %s" % tweet_url(reply))
                yield reply
                # recursive magic to also get the replies to this reply
                for reply_to_reply in get_replies(reply):
                    yield reply_to_reply
            max_id = reply.id
        if len(replies) != 100:
            break

if __name__ == "__main__":
    #status = t.PostUpdate('Test tweet from kMd lab')
    #print(status.id)
    #id = str(status.id)
    #with open('tweets.data', 'a') as the_file:
      #the_file.writelines('\n{"user":{"screen_name":"'+status.user.screen_name+'"},"id":'+ id +'}')
    logging.basicConfig(filename="replies.log", level=logging.INFO)
    tweets_file = "tweets.data" 
    for tweet in get_tweets(tweets_file):
        for reply in get_replies(tweet):
            #print(reply.AsJsonString())
            print(reply.text +" by "+ reply.user.name)
            

@_Majed_Ali positive by test2
@_Majed_Ali positive by test2
@_Majed_Ali 2 by Myan Le
@_Majed_Ali 3 by Ahmad Abdullah Tariq
@_Majed_Ali to be continued.... by Myan Le
@_Majed_Ali I am commenting your another tweet from Python... by Myan Le
@_Majed_Ali Cool! by Myan Le
@_Majed_Ali Fighting! :D by Myan Le
@_Majed_Ali to be continued.... by Myan Le
